In [ ]:
from datasets import load_dataset

In [ ]:
rdata=load_dataset('glue','sst2')
rdata

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sst2/train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

sst2/validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

sst2/test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [ ]:
rdata['train'][10]

{'sentence': 'goes to absurd lengths ', 'label': 0, 'idx': 10}

In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [ ]:
tok=AutoTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokens=tok(
    list(rdata['train']['sentence']),
    truncation=True
)

In [ ]:
def tok_fn(batch):
  return tok(
    list(batch['sentence']),
    truncation=True
)

In [ ]:
tok_data=rdata.map(tok_fn,batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
tok_data

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
col=DataCollatorWithPadding(tokenizer=tok)

In [ ]:
from transformers import TrainingArguments

In [ ]:
train_agu=TrainingArguments('train-test',eval_strategy='epoch',
                            fp16=True,
                            per_device_train_batch_size=4,
                            gradient_accumulation_steps=4,
                            learning_rate=3e-5,
                            num_train_epochs=2)

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 567.6 kB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
import numpy as np

In [ ]:
def evmetric(eval_preds):
  metric=evaluate.load('glue','sst2')
  logits,labels=eval_preds
  pred=np.argmax(logits,axis=-1)
  return metric.compute(predictions=pred,references=labels)

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    train_agu,
    train_dataset=tok_data["train"],
    eval_dataset=tok_data["validation"],
    data_collator=col,
    processing_class=tok,
    compute_metrics=evmetric,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bhalaniakshat (bhalaniakshat-dwarkadas-j-sanghvi-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.176100,0.228169,0.930046
2,0.112700,0.270082,0.931193


TrainOutput(global_step=8420, training_loss=0.1662769657415902, metrics={'train_runtime': 1859.185, 'train_samples_per_second': 72.45, 'train_steps_per_second': 4.529, 'total_flos': 1658623315217760.0, 'train_loss': 0.1662769657415902, 'epoch': 2.0})

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `basic_sentimentanalysis_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Cannot aut

In [ ]:
trainer.push_to_hub("basic_sentimentanalysis_finetuning_sst2")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /content/train-test/model.safetensors :   0%|          | 14.2kB /  438MB            

  ...vents.1757705262.ffa25e8af991.674.0:   2%|1         |   157B / 9.34kB            

  /content/train-test/training_args.bin :   2%|1         |  97.0B / 5.78kB            

CommitInfo(commit_url='https://huggingface.co/ByteMeHarder-404/train-test/commit/845296bd06f5ce81665c1087372420b5ab54fc6a', commit_message='basic_sentimentanalysis_finetuning_sst2', commit_description='', oid='845296bd06f5ce81665c1087372420b5ab54fc6a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ByteMeHarder-404/train-test', endpoint='https://huggingface.co', repo_type='model', repo_id='ByteMeHarder-404/train-test'), pr_revision=None, pr_num=None)